In [1]:
import pandas as pd

# Step 1: Define standard fields with requirement levels and justifications
validated_fields = [
    ("FAIR", "dc:title", "✅ Core", "Essential for discovery (DCMI)"),
    ("FAIR", "dc:description", "✅ Core", "Helps users interpret dataset meaning"),
    ("FAIR", "dc:creator", "✅ Core", "Attribution required by FAIR"),
    ("FAIR", "dc:license", "✅ Core", "Required for reuse clarity"),
    ("FAIR", "dc:publisher", "🟡 Recommended", "Common in DCAT and citation systems"),
    ("FAIR", "dc:subject", "🟡 Recommended", "Improves search and categorization"),
    ("FAIR", "dc:issued", "🟡 Recommended", "Standard publishing metadata"),
    ("FAIR", "dc:language", "🟤 Optional", "Relevant for multilingual datasets"),
    ("FAIR", "dcat:landingPage", "✅ Core", "Required by DCAT for linking dataset access"),

    ("PROV-O", "prov:Entity", "✅ Core", "Describes the dataset or model as an object"),
    ("PROV-O", "prov:Activity", "✅ Core", "Connects actions to data objects"),
    ("PROV-O", "prov:Agent", "✅ Core", "Identifies responsible party"),
    ("PROV-O", "prov:wasGeneratedBy", "✅ Core", "Links output to activity"),
    ("PROV-O", "prov:used", "✅ Core", "Links input to activity"),
    ("PROV-O", "prov:wasAssociatedWith", "🟡 Recommended", "Useful for complex systems"),
    ("PROV-O", "prov:startedAtTime", "✅ Core", "Supports reproducibility"),
    ("PROV-O", "prov:endedAtTime", "✅ Core", "Supports reproducibility"),

    ("FAIR4ML", "fair4ml:trainedOn", "✅ Core", "Links model to dataset"),
    ("FAIR4ML", "fair4ml:trainingStartTime", "✅ Core", "Temporal traceability"),
    ("FAIR4ML", "fair4ml:trainingEndTime", "✅ Core", "Temporal traceability"),
    ("FAIR4ML", "fair4ml:modelType", "✅ Core", "Essential for reuse and understanding"),
    ("FAIR4ML", "fair4ml:targetVariable", "✅ Core", "Key for supervised learning"),
    ("FAIR4ML", "fair4ml:trainingScriptVersion", "✅ Core", "Links model to source code"),
    ("FAIR4ML", "fair4ml:runEnvironment", "🟡 Recommended", "Helpful for reproduction"),

    ("MLSEA", "mlsea:accuracy", "✅ Core", "Primary performance metric"),
    ("MLSEA", "mlsea:f1_score", "✅ Core", "Widely used for imbalance"),
    ("MLSEA", "mlsea:roc_auc", "🟡 Recommended", "For probabilistic classifiers"),
    ("MLSEA", "mlsea:precision", "🟡 Recommended", "Used in evaluation"),
    ("MLSEA", "mlsea:recall", "🟡 Recommended", "Used in evaluation"),

    ("Croissant", "mls:modelName", "✅ Core", "Essential identifier"),
    ("Croissant", "mls:learningAlgorithm", "✅ Core", "Defines the method used"),
    ("Croissant", "mls:hyperparameters", "🟡 Recommended", "Enhances reproducibility"),
    ("Croissant", "mls:hasInput", "✅ Core", "Links to input dataset"),
    ("Croissant", "mls:hasOutput", "🟡 Recommended", "Expected result structure"),

    ("Internal", "session_metadata.username", "✅ Core", "Links action to a person"),
    ("Internal", "session_metadata.role", "🟡 Recommended", "Improves team attribution"),
    ("Internal", "git_metadata.commit_hash", "✅ Core", "Traceability to exact code version"),
    ("Internal", "git_metadata.branch", "🟡 Recommended", "Clarifies versioning"),
    ("Internal", "justification.why_model", "🟡 Recommended", "Supports explainability"),
    ("Internal", "justification.why_dataset", "🟡 Recommended", "Supports explainability"),
]

# Assign weights
weight_lookup = {"✅ Core": 1.0, "🟡 Recommended": 0.5, "🟤 Optional": 0.25}

# Create DataFrame
validated_df = pd.DataFrame([
    {
        "Standard": std,
        "Field": field,
        "Requirement": level,
        "Weight": weight_lookup.get(level, 0),
        "Justification": just
    } for std, field, level, just in validated_fields
])


In [3]:
import json

# Load structured metadata from your run
with open("../MODEL_PROVENANCE/RandomForest_Iris_v20250616_154241/structured_metadata.json", "r") as f:
    structured_metadata = json.load(f)

# Flatten all fields
flat_fields = set()
for section, fields in structured_metadata.items():
    if isinstance(fields, dict):
        for key in fields:
            if isinstance(fields[key], dict):
                for subkey in fields[key]:
                    flat_fields.add(f"{key}.{subkey}")
            else:
                flat_fields.add(key)


In [4]:
# Compare and score
total_possible = 0
total_achieved = 0
comparison = []

for _, row in validated_df.iterrows():
    field = row["Field"]
    found = "✅" if field in flat_fields else "❌"
    score = row["Weight"] if found == "✅" else 0
    total_possible += row["Weight"]
    total_achieved += score
    comparison.append({
        "Standard": row["Standard"],
        "Field": field,
        "Requirement": row["Requirement"],
        "Captured?": found,
        "Score": score,
        "Max Score": row["Weight"],
        "Justification": row["Justification"]
    })

# Create results DataFrame and summary
comparison_df = pd.DataFrame(comparison)

print("🔎 Metadata Coverage Summary")
print(f"Score: {total_achieved} / {total_possible}")
print(f"Coverage: {round((total_achieved / total_possible) * 100, 2)}%")
comparison_df.head()  # Display first few rows


🔎 Metadata Coverage Summary
Score: 15.0 / 32.25
Coverage: 46.51%


,Standard,Field,Requirement,Captured?,Score,Max Score,Justification
0,FAIR,dc:title,✅ Core,✅,1.0,1.0,Essential for discovery (DCMI)
1,FAIR,dc:description,✅ Core,✅,1.0,1.0,Helps users interpret dataset meaning
2,FAIR,dc:creator,✅ Core,✅,1.0,1.0,Attribution required by FAIR
3,FAIR,dc:license,✅ Core,✅,1.0,1.0,Required for reuse clarity
4,FAIR,dc:publisher,🟡 Recommended,❌,0.0,0.5,Common in DCAT and citation systems
